In [1]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', sep=',', header=None, names=['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','class'])

df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
df = df.rename(columns={'class': 'household_income'})

In [2]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
conn = engine.connect()

from sqlalchemy import Column, Integer, String

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

2018-04-03 10:46:41,460 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-03 10:46:41,460 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:41,460 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-03 10:46:41,473 INFO sqlalchemy.engine.base.Engine ()


In [3]:
class Adults(Base):
    __tablename__ = 'SocioEconomicStatus'

    Id = Column(Integer, primary_key = True)
    age = Column(Integer)
    workclass = Column(String)
    fnlwgt = Column(Integer)
    education = Column(String)
    education_num = Column(Integer)
    marital_status = Column(String)
    occupation = Column(String)
    relationship = Column(String)
    race = Column(String)
    sex = Column(String)
    capital_gain = Column(Integer)
    capital_loss = Column(Integer)
    hours_per_week = Column(Integer)
    native_country = Column(String)
    household_income = Column(String)

    def __repr__(self):
        return """<User(age='%d', workclass='%s', 
                        fnlwgt='%d', education='%s', 
                        education_num='%d', marital_status='%s', 
                        occupation='%s', relationship='%s', 
                        race='%s', sex='%s', capital_gain='%d',
                        capital_loss='%d', hours_per_week='%d', 
                        native_country='%s', household_income='%s')>""" % (
                        self.age, self.workclass, self.fnlwgt,
                        self.education, self.education_num, self.marital_status,
                        self.occupation, self.relationship, self.race,
                        self.sex, self.capital_gain, self.capital_loss,
                        self.hours_per_week, self.native_country, self.household_income)

In [4]:
Adults.__table__ 

Table('SocioEconomicStatus', MetaData(bind=None), Column('Id', Integer(), table=<SocioEconomicStatus>, primary_key=True, nullable=False), Column('age', Integer(), table=<SocioEconomicStatus>), Column('workclass', String(), table=<SocioEconomicStatus>), Column('fnlwgt', Integer(), table=<SocioEconomicStatus>), Column('education', String(), table=<SocioEconomicStatus>), Column('education_num', Integer(), table=<SocioEconomicStatus>), Column('marital_status', String(), table=<SocioEconomicStatus>), Column('occupation', String(), table=<SocioEconomicStatus>), Column('relationship', String(), table=<SocioEconomicStatus>), Column('race', String(), table=<SocioEconomicStatus>), Column('sex', String(), table=<SocioEconomicStatus>), Column('capital_gain', Integer(), table=<SocioEconomicStatus>), Column('capital_loss', Integer(), table=<SocioEconomicStatus>), Column('hours_per_week', Integer(), table=<SocioEconomicStatus>), Column('native_country', String(), table=<SocioEconomicStatus>), Column(

In [5]:
Base.metadata.create_all(engine)

2018-04-03 10:46:41,740 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("SocioEconomicStatus")
2018-04-03 10:46:41,742 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:41,746 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "SocioEconomicStatus" (
	"Id" INTEGER NOT NULL, 
	age INTEGER, 
	workclass VARCHAR, 
	fnlwgt INTEGER, 
	education VARCHAR, 
	education_num INTEGER, 
	marital_status VARCHAR, 
	occupation VARCHAR, 
	relationship VARCHAR, 
	race VARCHAR, 
	sex VARCHAR, 
	capital_gain INTEGER, 
	capital_loss INTEGER, 
	hours_per_week INTEGER, 
	native_country VARCHAR, 
	household_income VARCHAR, 
	PRIMARY KEY ("Id")
)


2018-04-03 10:46:41,747 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:41,750 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
### Load data from the dataframe to the table - all rows ### 

from sqlalchemy.orm import sessionmaker

tableToWriteTo = 'SocioEconomicStatus'

#create Index column to use as primary key
df.reset_index(inplace=True)
df.rename(columns={'index':'Id'}, inplace =True)

listToWrite = df.to_dict(orient='records')

metadata = sqlalchemy.schema.MetaData(bind=engine)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

# Open the session
Session = sessionmaker(bind=engine)
session = Session()

# Inser the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()

2018-04-03 10:46:42,610 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("SocioEconomicStatus")
2018-04-03 10:46:42,610 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:42,616 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'SocioEconomicStatus' AND type = 'table'
2018-04-03 10:46:42,618 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:42,622 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("SocioEconomicStatus")
2018-04-03 10:46:42,623 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:42,625 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'SocioEconomicStatus' AND type = 'table'
2018-04-03 10:46:42,627 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 10:46:42,632 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("SocioEconomicStatus")
2018-04-03 10:46:42,634 INFO sqlalch

In [7]:
### UPDATE QUERIES ### 

from sqlalchemy import and_, or_, not_

session.query(Adults).filter( or_(
        Adults.education == '7th-8th',
        Adults.education == '9th',
        Adults.education == '10th'
    )).update({"education": 'Junior High School'})
  
session.commit()

session.query(Adults).filter( or_(
        Adults.education == '1st-4th',
        Adults.education == '5th-6th',
    )).update({"education": 'Primary School'})
  
session.commit()

2018-04-03 10:46:43,280 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-03 10:46:43,282 INFO sqlalchemy.engine.base.Engine UPDATE "SocioEconomicStatus" SET education=? WHERE "SocioEconomicStatus".education = ? OR "SocioEconomicStatus".education = ? OR "SocioEconomicStatus".education = ?
2018-04-03 10:46:43,284 INFO sqlalchemy.engine.base.Engine ('Junior High School', '7th-8th', '9th', '10th')
2018-04-03 10:46:43,299 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-03 10:46:43,304 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-03 10:46:43,306 INFO sqlalchemy.engine.base.Engine UPDATE "SocioEconomicStatus" SET education=? WHERE "SocioEconomicStatus".education = ? OR "SocioEconomicStatus".education = ?
2018-04-03 10:46:43,308 INFO sqlalchemy.engine.base.Engine ('Primary School', '1st-4th', '5th-6th')
2018-04-03 10:46:43,318 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
### DELETE QUERIES ### 
session.query(Adults).filter(Adults.workclass == '?').delete()

session.query(Adults).filter(Adults.occupation == '?').delete()

2018-04-03 10:46:43,423 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-03 10:46:43,423 INFO sqlalchemy.engine.base.Engine DELETE FROM "SocioEconomicStatus" WHERE "SocioEconomicStatus".workclass = ?
2018-04-03 10:46:43,423 INFO sqlalchemy.engine.base.Engine ('?',)
2018-04-03 10:46:43,439 INFO sqlalchemy.engine.base.Engine DELETE FROM "SocioEconomicStatus" WHERE "SocioEconomicStatus".occupation = ?
2018-04-03 10:46:43,441 INFO sqlalchemy.engine.base.Engine ('?',)


7

In [9]:
### FILTER QUERY - count adults with a masters degrees and a private job ##
session.query(Adults).filter( and_(
        Adults.workclass=='Private',
        Adults.education=='Masters'
    )).count()

2018-04-03 10:46:43,504 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT "SocioEconomicStatus"."Id" AS "SocioEconomicStatus_Id", "SocioEconomicStatus".age AS "SocioEconomicStatus_age", "SocioEconomicStatus".workclass AS "SocioEconomicStatus_workclass", "SocioEconomicStatus".fnlwgt AS "SocioEconomicStatus_fnlwgt", "SocioEconomicStatus".education AS "SocioEconomicStatus_education", "SocioEconomicStatus".education_num AS "SocioEconomicStatus_education_num", "SocioEconomicStatus".marital_status AS "SocioEconomicStatus_marital_status", "SocioEconomicStatus".occupation AS "SocioEconomicStatus_occupation", "SocioEconomicStatus".relationship AS "SocioEconomicStatus_relationship", "SocioEconomicStatus".race AS "SocioEconomicStatus_race", "SocioEconomicStatus".sex AS "SocioEconomicStatus_sex", "SocioEconomicStatus".capital_gain AS "SocioEconomicStatus_capital_gain", "SocioEconomicStatus".capital_loss AS "SocioEconomicStatus_capital_loss", "SocioEconomicStatus".hours_pe

894

In [10]:
### FILTER QUERY - Married Males###

session.query(Adults).filter( and_(
        Adults.marital_status.like('%Married%'),
        Adults.sex == 'Male'
    )).count()

2018-04-03 10:46:43,592 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT "SocioEconomicStatus"."Id" AS "SocioEconomicStatus_Id", "SocioEconomicStatus".age AS "SocioEconomicStatus_age", "SocioEconomicStatus".workclass AS "SocioEconomicStatus_workclass", "SocioEconomicStatus".fnlwgt AS "SocioEconomicStatus_fnlwgt", "SocioEconomicStatus".education AS "SocioEconomicStatus_education", "SocioEconomicStatus".education_num AS "SocioEconomicStatus_education_num", "SocioEconomicStatus".marital_status AS "SocioEconomicStatus_marital_status", "SocioEconomicStatus".occupation AS "SocioEconomicStatus_occupation", "SocioEconomicStatus".relationship AS "SocioEconomicStatus_relationship", "SocioEconomicStatus".race AS "SocioEconomicStatus_race", "SocioEconomicStatus".sex AS "SocioEconomicStatus_sex", "SocioEconomicStatus".capital_gain AS "SocioEconomicStatus_capital_gain", "SocioEconomicStatus".capital_loss AS "SocioEconomicStatus_capital_loss", "SocioEconomicStatus".hours_pe

18565

In [11]:
### FUNCTION QUERY - count(*) ###

from sqlalchemy import func

session.query(func.count('*')).select_from(Adults).scalar()

2018-04-03 10:46:43,670 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM "SocioEconomicStatus"
2018-04-03 10:46:43,672 INFO sqlalchemy.engine.base.Engine ('*',)


30718

In [12]:
### FUNCTION QUERY - group adults by workclass ###

session.query(Adults.workclass, func.count(Adults.Id)).group_by(Adults.workclass).all()

2018-04-03 10:46:43,758 INFO sqlalchemy.engine.base.Engine SELECT "SocioEconomicStatus".workclass AS "SocioEconomicStatus_workclass", count("SocioEconomicStatus"."Id") AS count_1 
FROM "SocioEconomicStatus" GROUP BY "SocioEconomicStatus".workclass
2018-04-03 10:46:43,758 INFO sqlalchemy.engine.base.Engine ()


[('Federal-gov', 960),
 ('Local-gov', 2093),
 ('Private', 22696),
 ('Self-emp-inc', 1116),
 ('Self-emp-not-inc', 2541),
 ('State-gov', 1298),
 ('Without-pay', 14)]